In [1]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=17e635bcc28eb362a0c1069b183bfda8db4760645eadb3a99dafa67d94c42e33
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [3]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [15]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [16]:
# if "COLAB_GPU" in os.environ:
#   !huggingface-cli login
# else:
#   notebook_login()

from huggingface_hub import login
login()


In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# base_model_id = "meta-llama/Llama-2-7b-chat-hf"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# tokenizer = AutoTokenizer.from_pretrained(base_model_id)


In [17]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [18]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 42.50 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [26]:
# from datasets import Dataset

# # Read the file line-by-line
# with open("/content/Fine-tuning-LLMs/data/hawaii_wf_4.txt", "r", encoding="utf-8") as f:
#     lines = f.readlines()

# # Wrap in a Hugging Face dataset
# train_dataset = Dataset.from_dict({"text": lines})

# # Verify the dataset
# print(train_dataset)


Dataset({
    features: ['text'],
    num_rows: 158
})


In [31]:
from datasets import Dataset

# Read the first file line-by-line
with open("/content/Fine-tuning-LLMs/data/hawaii_wf_4.txt", "r", encoding="utf-8") as f1:
    lines1 = f1.readlines()

# Read the second file line-by-line
with open("/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt", "r", encoding="utf-8") as f2:
    lines2 = f2.readlines()

# Combine both file contents
all_lines = lines1 + lines2

# Wrap in a Hugging Face dataset
train_dataset = Dataset.from_dict({"text": all_lines})

# Verify the dataset
print(train_dataset)


Dataset({
    features: ['text'],
    num_rows: 241
})


In [32]:
train_dataset["text"][0]

'In the early morning of August 9, 2023, the officer further assisted in coordinating transportation for people who\n'

In [34]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [35]:
tokenized_train_dataset = []
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [36]:
tokenized_train_dataset[1]

{'input_ids': [1, 750, 4586, 25447, 297, 278, 23474, 304, 10169, 278, 3974, 29892, 5662, 3864, 896, 7450, 278, 11176, 14703, 27709, 23511, 29889, 13, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [37]:
tokenized_train_dataset[2]

{'input_ids': [1, 28288, 16535, 29871, 29947, 13, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [38]:
tokenizer.eos_token

'</s>'

In [39]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [41]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=20,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saurabhdongre8767 (saurabhdongre8767-gh-raisoni-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.649900
20,3.357400


TrainOutput(global_step=20, training_loss=3.5036474227905274, metrics={'train_runtime': 211.0933, 'train_samples_per_second': 0.379, 'train_steps_per_second': 0.095, 'total_flos': 93206798598144.0, 'train_loss': 3.5036474227905274, 'epoch': 0.3305785123966942})

In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
  )


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True

                              )

modelFinetuned = PeftModel.from_pretrained(base_model, "finetunedModel/checkpoint-20")

In [47]:
user_question = "Who is Narendra Modi?"

eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: Who is Narendra Modi? Just answer this question accurately and concisely.

Answer: Narendra Modi is the 14th and current Prime Minister of India, serving since 2014. He is the leader of the Bharatiya Janata Party (BJP) and has been instrumental in shaping India's political landscape. Modi was born on September 17, 1950, in Vadnagar, Gujarat, and began his political career in the RSS (Rashtriya Swayamsevak Sangh) before joining the BJP. He served as the Chief Minister of Gujarat from 2001 to 2014, implementing various development projects and initiatives that contributed to the state's rapid growth. Modi's election as Prime Minister in 2014 marked a significant shift in India's political landscape, as he became the first Prime Minister from the BJP to hold the office since 1984. Under his leadership, the government has implemented several policies and initiatives aimed at promoting economic growth, improving infrastructure, and enhancing India's global stature. Modi's tenure h

In [48]:
!zip -r train_data.zip /content/finetunedModel

  adding: content/finetunedModel/ (stored 0%)
  adding: content/finetunedModel/checkpoint-20/ (stored 0%)
  adding: content/finetunedModel/checkpoint-20/rng_state.pth (deflated 25%)
  adding: content/finetunedModel/checkpoint-20/optimizer.pt (deflated 10%)
  adding: content/finetunedModel/checkpoint-20/tokenizer.model (deflated 55%)
  adding: content/finetunedModel/checkpoint-20/adapter_model.safetensors (deflated 7%)
  adding: content/finetunedModel/checkpoint-20/tokenizer_config.json (deflated 66%)
  adding: content/finetunedModel/checkpoint-20/adapter_config.json (deflated 56%)
  adding: content/finetunedModel/checkpoint-20/scheduler.pt (deflated 56%)
  adding: content/finetunedModel/checkpoint-20/README.md (deflated 66%)
  adding: content/finetunedModel/checkpoint-20/trainer_state.json (deflated 57%)
  adding: content/finetunedModel/checkpoint-20/special_tokens_map.json (deflated 79%)
  adding: content/finetunedModel/checkpoint-20/training_args.bin (deflated 52%)
